# Clusterverfahren

In diesem Kapitel geht es um "nicht überwachtes Lernen". Wir werden zwei Clusterverfahren kennenlernen: *k-Means* und *Agglomeratives Hierarchisches Linkage Average Clustering*.

Wir beginnen mit k-Means-Clustering.

## k-Means Clustering

In [ ]:
if(!require("ggplot2")) install.packages("ggplot2")
if(!require("ggrepel")) install.packages("ggrepel")
if(!require("dplyr")) install.packages("dplyr")
if(!require("gapminder")) install.packages("gapminder")
if(!require("factoextra")) install.packages("factoextra")

Wir beginnen mit einem einfachen Beispiel, das das Prinzip des k-Means-Algorithmus verdeutlicht. Zuerst erstellen wir ein Dataframe.

In [ ]:
daten <- data.frame(Attribut1=c(1,2,4,5),
                    Attribut2=c(1,1,3,4))

daten
plot(daten)

Wir erstellen ein Modell mit Hilfe der Funktion *km*. Wir setzen den Hyperparameter k auf 2, da wir 2 Cluster bilden wollen:

In [ ]:
km <- kmeans(daten, 2)

Wir können nun bereits die Koordinaten der Center-Punkte ausgeben:

In [ ]:
print(km$centers)

Zur Verdeutlichung plotten wir die Punkte nochmals. Wir färben die Punkte abhängig von ihrem zugeordneten Cluster ein. Zusätzlich fügen wir die Centerpunkte ein:

In [ ]:
plot(daten, pch=0, cex=2, col=km$cluster)
points(km$centers, pch=17, cex=2, col="green")

## Beispiel mit dem Datensatz *IRIS*

Im folgenden Beispiel wenden wir dies auf den IRIS-Datensatz an. Wir erstellen 3 Cluster, da es 3 verschiedene IRIS-Typen gibt. Wir färben die Punkte wieder abhängig von ihrem Cluster ein. Da wir einen 2D-Plot ausgeben, können wir nicht alle Features berücksichtigen (das wären 4 Dimensionen). Daher sind einige Beobachtungen scheinbar nicht optimal klassifiziert. Jedenfalls erkenne wir, dass es einen Überlappungsbereich zwischen den Spezies Versicolor und Virginica gibt, während Setosa sehr gut separiert werden kann.

In [ ]:
model.iris <- kmeans(iris[,1:4],3)

c <- data.frame(model.iris$centers)
centers <- data.frame(x=c$Petal.Length, y=c$Petal.Width)

iris %>% ggplot(aes(Petal.Length,Petal.Width, color=iris$Species)) + geom_point() +
  geom_point(data=centers, aes(x=x, y=y, color="CENTER")) 

## Beispiel mit dem Datensatz *mtcars*

Im folgenden Beispiel clustern wir die Fahrzeuge aus *mtcars* bzgl. ihres Verbrauchs (*mpg*) und Gewicht (*weight*):

In [ ]:
#Datenframe erstellen und standardisieren
datenkm <- data.frame(Verbrauch=mtcars$mpg, Gewicht=mtcars$wt)
datenkm <- data.frame(scale(datenkm))


#Clustern nach dem k-means-Algorithmus, es sollen 4 Cluster erstellt werden
cl <- kmeans(data.matrix(datenkm), 4)


#Grafik plotten
ggplot(datenkm, aes(x=Gewicht, y=Verbrauch)) +
  geom_point(col=cl$cluster, size=4, alpha=0.5) +
  geom_text_repel(label=row.names(mtcars)) + ggtitle("K-means Cluster")

## Beispiel mit Datensatz *USArrest*

Im folgenden Beispiel wollen wir US-Bundesstaaten bzgl. ihrer schweren Straftaten clustern!

In [ ]:
head(USArrests)
model.arrest <- eclust(scale(USArrests), "kmeans", k=4, graph = F)
fviz_cluster(model.arrest, ellipse.type = "norm", ellipse.level = 0.68, main = "US-Bundesstaaten nach Straftaten")

## Beispiel: Lebenserwartung in Ländern

In diesem Beispiel clustern wir Länder (mit > 30 Mill. Einwohnern) bzgl. der Lebenswerartung und Jahreseinkommen.

In [ ]:
data("gapminder")

#Binden (direkter Zugriff auf die Attribute ohne Angabe des data.frames)
attach(gapminder)



#Datenframe erstellen, filtern auf Länder ab 30 Mio. Einwohnern
daten <- data.frame(land=country, gdpPercap, lifeExp) %>% filter(pop>30e6)

#Daten gruppieren: Mittelwerte über die Jahre bilden
daten <- daten %>% group_by(land) %>% summarise(Einkommen=mean(gdpPercap), Lebenserwartung=mean(lifeExp))

#Clustern!
cl = kmeans(data.matrix(daten), 5)

#Grafik plotten, Punkte in Größe 4 mit Tranparenz, Farbe je noch Cluster, Punkte mit Ländernamen beschriften
daten %>% ggplot(aes(x=Einkommen, y=Lebenserwartung, label=land)) + 
  geom_point(col=cl$cluster,size=4, alpha=0.5) +
  geom_text_repel()


detach(gapminder)